In [1]:
from evaluation import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
preprocessor = lambda text: stem(remove_tags(chiffre_suppression(ponc_suppression(text))))
vect_params = {'lowercase':True,'strip_accents':'ascii','stop_words':'english'}

model_params = {'random_state':0}
predictions = prediction_generator(preprocessor=preprocessor,
                                   vectorizer=CountVectorizer,
                                   vect_params=vect_params,
                                   model=LinearSVC,
                                   model_params=model_params,
                                   save=True)


C:\Users\PC\miniconda3\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alway', 'anoth', 'anyon', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'describ', 'don', 'du', 'eith', 'elsewher', 'everyon', 'everywher', 'fir', 'fiv', 'form', 'furth', 'giv', 'hav', 'henc', 'hereaft', 'howev', 'latt', 'mad', 'meanwhil', 'min', 'mor', 'moreov', 'mov', 'nam', 'neith', 'nev', 'nin', 'non', 'noon', 'nowher', 'otherwis', 'ourselv', 'perhap', 'pleas', 'rath', 'sam', 'se', 'sid', 'sinc', 'sincer', 'som', 'someon', 'sometim', 'somewher', 'tak', 'themselv', 'thenc', 'ther', 'thereaft', 'therefor', 'thes', 'thos', 'thre', 'togeth', 'twelv', 'vi', 'wel', 'wer', 'whatev', 'whenc', 'whenev', 'wher', 'whereaft', 'wherev', 'wheth', 'whil', 'whith', 'whoev', 'whol', 'whos', 'yourselv'] not in stop_words.
  warnings.warn(
C:\Users\PC\miniconda3\Li